<a href="https://colab.research.google.com/github/SchachtmanLab/Transgenic-sorghum-sorgoleone/blob/master/%5BSTUDENT_COPY%5DThursday_afternoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration with DataFrames, cont.

<font color = '#ed865c' size = 4>**Make sure to press the play button to run the cell below: this will re-load the datasets and functions that we worked on during the last section.**</font>

In [31]:
#@title
# make sure to run this cell to import the external files we need for today
# and load in the appropriate packages
!git clone https://github.com/ccbskillssem/pythonbootcamp.git

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

alleles = pd.read_table('/content/pythonbootcamp/day_4/alleles.tsv')
clinvar = pd.read_table('/content/pythonbootcamp/day_4/clinvar.tsv')

fatal: destination path 'pythonbootcamp' already exists and is not an empty directory.


## Changing labels

<font color = '#ed865c' size = 4>**Make sure to run the cell above before you begin: this will re-load the datasets and functions that we worked on during the last section.**</font>

### Row labels

Currently, the row labels for `alleles` are numeric, which makes it easy to index rows just as we did with arrays. However, it's also possible to use row *names* as labels, just as we do with columns.

To do this, we can select an existing column of the DataFrame and turn it into the row Index. However, it's **extremely** important that we make sure the candidate column contains only unique values: we wouldn't want to try and index a row name, only to retrieve multiple rows with the same name!

Series have a useful attribute called `.is_unique`, which is `True` if all values in the Series are unique. Given that columns of DataFrames are stored as Series, we can easily check our candidate columns.

In [32]:
alleles.chromosome.is_unique
alleles.columns


Index(['chromosome', 'position', 'ID', 'reference', 'alternative', 'quality',
       'filter', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF'],
      dtype='object')

In [10]:
# try it out:
# print the .is_unique attribute of each column
alleles.chromosome.is_unique
for col in alleles.columns:
  print(alleles[col].is_unique)
# hint: remember for loops? try using a loop with the column labels


False
False
True
False
False
False
False
False
False
False
False
False
False


Great – looks like there's a candidate column for our row labels. We can now change `allele`'s row labels using the `.set_index()` method.

In [33]:
# in this case, we're opting to update the variable

alleles = alleles.set_index('ID')
alleles.head()

,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
ID,,,,,,,,,,,,
rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.0,1.000
rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.0,0.000
rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.0,0.001


Excellent! As you can see, each row's `ID` value is now the row label. We can access each row using `.loc[]` and the corresponding `ID` label.

In [34]:
alleles.loc['rs573303859']

,rs573303859
chromosome,13
position,100119866
reference,A
alternative,ATTAT
quality,93.0
filter,PASS
AF,0.001198
AFR_AF,0.0038
AMR_AF,0.0
EAS_AF,0.0


Row names can be convenient, but also cumbersome: for example, now that we've assigned `ID` as our row labels, all of the row labels are strings: thus, we can no longer slice multiple rows with `.loc[]`.

> If you really want to slice multiple rows *and* retain string-type row labels, you will need to use `.iloc[]` instead of `.loc[]`. `.iloc[]` is used to access columns/rows/slices/cells in a `numpy` array-like manner. You can read more about it [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-position).

In [35]:
# try it out:
# access the row labeled rs563299082
alleles.loc['rs563299082']
# and its chromosome, position, reference, and alternative values
alleles.loc['rs563299082'][0:4]
alleles.loc['rs563299082'].iloc[0:4]
alleles.loc['rs563299082', ['chromosome','position','reference','alternative']]

,rs563299082
chromosome,22
position,30628242
reference,C
alternative,A


If you end up wanting to revert back to the standard 0-index numeric row labels, you can use the `.reset_index()` method to convert the row names back to a regular column.

In [36]:
# let's return alleles to its original state

alleles = alleles.reset_index()
alleles.head()
#alleles = alleles.reset_index(inplace=True)

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
0,rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.0,1.000
1,rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.0,0.000
2,rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
3,rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
4,rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.0,0.001


### Column labels
In the case that we want to replace column labels, we can use the `.rename()` method, which is the equivalent of `.replace()` for column labels. Just as we showed you before, the easiest way to do this is to provide a dictionary of target and substitute labels.

For example, let's consider `reference` and `alternative`, which are commonly shortened to `REF` and `ALT` in genomic file formats. Changing the column labels while maintaining the column content is a single-line operation:

In [37]:
# rename reference and alternative to REF and ALT for all rows

alleles.rename({'reference': 'REF',
                 'alternative': 'ALT'}, axis = 'columns').head()

,ID,chromosome,position,REF,ALT,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
0,rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.0,1.000
1,rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.0,0.000
2,rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
3,rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
4,rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.0,0.001


## Sorting by column(s)
There's no way to sugar-coat it: there was no intuitive way for us to sort arrays by specific columns in `numpy`.

With `pandas`, the `.sort_values()` method allows us to sort a DataFrame/Series with the following inputs:
* A single column name, or a list of column names
* The `ascending` parameter, which takes a single Boolean value or a list of Boolean values (corresponding to the list of column names).
  * `True` will sort by lowest to highest (ascending).
  * `False` will sort by highest to lowest (descending).

In [40]:
# let's sort AFR_AF by highest to lowest frequency

alleles.sort_values("AFR_AF", ascending=False).head()  # low to high

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
73291,rs1898461,10,123051473,C,G,96.0,PASS,0.964457,1.0,0.9769,0.8621,0.9861,0.9908
45621,rs1340962,6,69854405,G,A,96.0,PASS,0.999201,1.0,0.9942,1.0000,1.0000,1.0000
138728,rs11464250,21,36866131,T,TA,92.0,PASS,1.000000,1.0,1.0000,1.0000,1.0000,1.0000
138669,rs59273667,21,36863854,GT,G,94.0,PASS,1.000000,1.0,1.0000,1.0000,1.0000,1.0000
32517,rs4835234,4,145602081,C,T,95.0,PASS,0.999800,1.0,1.0000,1.0000,0.9990,1.0000


⏸ **Exercise 1** Try sorting `alleles` by *all three* of the following columns:
* `AFR_AF`, from highest to lowest.
* `AMR_AF`, from lowest to highest.
* `EAS_AF`, from highest to lowest.

In [59]:
### write your code below ###
alleles.sort_values("AFR_AF", ascending=True)
alleles.sort_values("AMR_AF", ascending=False)
alleles.sort_values("EAS_AF", ascending=True)

a = alleles.sort_values("AFR_AF", ascending=True)
b = a.sort_values("AMR_AF", ascending=False)
print(b.sort_values("EAS_AF", ascending=True).head())

print(alleles.sort_values(["AFR_AF","AMR_AF","EAS_AF"], ascending=[True, False, True]).head())




                 ID  chromosome   position reference alternative  quality  \
132387  rs555444573          19   45572680         C           A     95.0   
138072  rs371485051          21   36843638         G           A     94.0   
20549   rs543015195           2  210633345         A           G     93.0   
27941   rs574793760           3  183095288         A           C     97.0   
105724  rs187205322          13  100341033         T           G     95.0   

       filter        AF  AFR_AF  AMR_AF  EAS_AF  EUR_AF  SAS_AF  
132387   PASS  0.000200  0.0000     0.0     0.0   0.000   0.001  
138072   PASS  0.000200  0.0008     0.0     0.0   0.000   0.000  
20549    PASS  0.000599  0.0023     0.0     0.0   0.000   0.000  
27941    PASS  0.000799  0.0023     0.0     0.0   0.001   0.000  
105724   PASS  0.000599  0.0023     0.0     0.0   0.000   0.000  
                ID  chromosome   position reference alternative  quality  \
178     rs45625835           1   11790137         C           T  

## Adding columns

We can easily create new columns by simply assigning a new column name and contents to a DataFrame.

In [61]:
# let's sample some rows with .sample()
# using 2024 as our seed value

some_alleles = alleles.sample(n = 5, random_state = 2024)
some_alleles

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
51950,rs574875069,6,80308094,A,G,95.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000
105482,rs565080472,13,100332270,G,T,93.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000
31588,rs565179676,4,108005855,G,A,92.0,PASS,0.000599,0.0000,0.0000,0.0,0.0000,0.0031
143874,rs550618366,21,46142330,G,A,94.0,PASS,0.000399,0.0015,0.0000,0.0,0.0000,0.0000
44323,rs141567447,6,69808224,G,A,94.0,PASS,0.009784,0.0015,0.0173,0.0,0.0219,0.0133


In [63]:
some_alleles['sample_col'] = 0 # a new column full of zeros
some_alleles['exp'] = "exp_1"
some_alleles

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,sample_col,exp
51950,rs574875069,6,80308094,A,G,95.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000,0,exp_1
105482,rs565080472,13,100332270,G,T,93.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000,0,exp_1
31588,rs565179676,4,108005855,G,A,92.0,PASS,0.000599,0.0000,0.0000,0.0,0.0000,0.0031,0,exp_1
143874,rs550618366,21,46142330,G,A,94.0,PASS,0.000399,0.0015,0.0000,0.0,0.0000,0.0000,0,exp_1
44323,rs141567447,6,69808224,G,A,94.0,PASS,0.009784,0.0015,0.0173,0.0,0.0219,0.0133,0,exp_1


---
⏸ **Exercise 2 (No coding)** What happens when you change the seed value of a random sampler (like `.sample()`)? What happens if you don't have a seed value specified?

Hint: One of the morning exercises might help you answer this question.

---

Next, we can also assign new columns by explicitly providing a column label and a "definition" for the column.

In [65]:
# let's reuse some_alleles
# creating a new column that contains the *reference* allele frequency
# this takes advantage of numpy broadcasting

some_alleles['reference_AF'] = 1 - some_alleles['AF']
some_alleles

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,sample_col,exp,reference_AF
51950,rs574875069,6,80308094,A,G,95.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000,0,exp_1,0.999800
105482,rs565080472,13,100332270,G,T,93.0,PASS,0.000200,0.0008,0.0000,0.0,0.0000,0.0000,0,exp_1,0.999800
31588,rs565179676,4,108005855,G,A,92.0,PASS,0.000599,0.0000,0.0000,0.0,0.0000,0.0031,0,exp_1,0.999401
143874,rs550618366,21,46142330,G,A,94.0,PASS,0.000399,0.0015,0.0000,0.0,0.0000,0.0000,0,exp_1,0.999601
44323,rs141567447,6,69808224,G,A,94.0,PASS,0.009784,0.0015,0.0173,0.0,0.0219,0.0133,0,exp_1,0.990216


Lastly, the `.map()` method allows us to **mutate** new columns based on values of existing columns. This is useful for adding categorical or Boolean columns without overwriting existing columns.

In [66]:
alleles['filter'].head()

,filter
0,PASS
1,PASS
2,PASS
3,PASS
4,PASS


In [73]:
# let's mutate a new column called filter_pass
# instead of replacing the values of filter

alleles['filter_pass'] = alleles['filter'].map({'PASS': True,
                                      'LowQ': False})
alleles.head()

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,filter_pass
0,rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.0,1.000,True
1,rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.0,0.000,True
2,rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000,True
3,rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000,True
4,rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.0,0.001,True


## Dropping columns

We can use the `.drop()` method to drop columns as easily as we created them. This can be handy if you end up with too many columns during your exploration, or if you simply don't need some of the columns that came with your dataset.

In [74]:
# getting rid of filter_pass
# removing it from all rows in the dataframe

#alleles = alleles.drop('filter_pass', axis='columns')
alleles = alleles.drop('filter_pass', axis=1)
alleles

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
0,rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.000,1.0000
1,rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.000,0.0000
2,rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.000,0.0000
3,rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.000,0.0000
4,rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.000,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146577,rs187829200,22,30628237,G,C,100.0,PASS,0.000200,0.0000,0.0000,0.001,0.000,0.0000
146578,rs563299082,22,30628242,C,A,97.0,PASS,0.000599,0.0000,0.0000,0.000,0.000,0.0031
146579,rs73398315,22,30628249,C,G,100.0,PASS,0.066494,0.2428,0.0159,0.000,0.001,0.0000
146580,rs370947798,22,30628260,C,T,100.0,PASS,0.000200,0.0008,0.0000,0.000,0.000,0.0000


## Grouping

We've now reached the operations that are *only* possible in `pandas`. **Grouped operations** partition the dataset into subsets based on the value of a given column or columns prior to operation.

There are two major steps in performing grouped operations:
1. **Splitting**: Splitting data into groups.
2. **Applying**: Applying a function across separate groups. (This is where vectorization is helpful!)

We can accomplish the first step using the `.group_by()` method, which takes one or more column labels.

In [83]:
alleles[alleles["filter"] == "LowQ"]
alleles[alleles["filter"] == "PASS"]
[alleles["filter"] == "LowQ"]
alleles.head()

,ID,chromosome,position,reference,alternative,quality,filter,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF
0,rs4845883,1,11784770,A,G,94.0,PASS,0.933506,0.7617,0.9741,1.000,1.0,1.000
1,rs139412854,1,11784774,C,T,96.0,PASS,0.000200,0.0008,0.0000,0.000,0.0,0.000
2,rs548376994,1,11784812,A,G,98.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
3,rs569782256,1,11784816,G,A,96.0,PASS,0.000200,0.0000,0.0000,0.001,0.0,0.000
4,rs537101792,1,11784829,G,A,95.0,PASS,0.000200,0.0000,0.0000,0.000,0.0,0.001


In [81]:
# this method doesn't show a visible output, it creates a groupby object

alleles.groupby('filter')

In order to see the effect of splitting with `.groupby()`, we need to chain `.groupby()` with a function that we want applied to our groups.

For example, let's say that we want to know the average quality score of alleles that pass (or don't pass) filter.

In [84]:
alleles.groupby('filter')['quality'].mean()
alleles.groupby('filter')['AF'].mean()
#

,AF
filter,
LowQ,0.032059
PASS,0.038074


## Merging (review)

This morning we discussed merging two datasets: `alleles` and `clinvar`. Merging is a very useful operation that we'll be using in tomorrow's final mini-project.

Once again, merge `alleles` and `clinvar` so that we have a single DataFrame called `merged`, which contains only the alleles present in both of the datsets.

In [100]:
# try it out:
# merge alleles and clinvar as you did earlier today into a 'merged' dataframe: inner join on the ID columns
mer_df_1 = pd.merge(alleles, clinvar, "inner", left_on='ID', right_on='ID')

mer_df_2 = pd.merge(alleles, clinvar, "inner", on = "ID")

mer_df_1 == mer_df_2
print(mer_df_1)
mer_df_1['chromosome_x'] == mer_df_1['chromosome_y']
(mer_df_1['chromosome_x'] == mer_df_1['chromosome_y']).all()
merged = mer_df_1
#merged = mer_df_1['AF']
#merged.head()

               ID  chromosome_x  position_x reference_x alternative_x  \
0      rs55740775             1    11786035           G            GT   
1      rs55780505             1    11786602       ATTTT             A   
2      rs55780505             1    11786602       ATTTT             A   
3      rs55780505             1    11786602       ATTTT             A   
4     rs547574100             1    11790673           C             T   
...           ...           ...         ...         ...           ...   
3744    rs2072195            22    30626746           A             T   
3745  rs187942607            22    30626821           T             G   
3746  rs149713328            22    30626902           T             G   
3747   rs56076607            22    30626903           C             T   
3748      rs10418            22    30626965           C             T   

      quality filter        AF  AFR_AF  AMR_AF  ...  SAS_AF  chromosome_y  \
0        92.0   PASS  0.951278  0.9221  0.9697

## Exporting

This is our final lesson in `pandas`: hooray!

We've learned a lot about how to parse and manipulate our DataFrames. At some point in the near future, you'll likely want to save the results of your operations.

The `.to_csv()` method allows us to easily export our DataFrame to a `.csv` file. As the name implies, it will save a DataFrame to a `.csv` file, given a string (file name).

Let's go ahead and cap off our lecture by exporting `merged` to a file called `merged.csv`.

In [99]:
# Reminder! Use this bash command to know which directory you are at to track location of saved files
!pwd

/content


In [118]:
merged.to_csv('/content/pythonbootcamp/day_4/my_first_merged.csv')
!pwd

/content/pythonbootcamp/day_4


In [117]:
# Try it out
# Read back in that same file to see how it was saved
reload = pd.read_csv('/content/my_first_merged.csv')
reload.head()

!pwd
%cd '/content/pythonbootcamp/day_4'
!pwd
#!pwd
#reload.head()

/content/pythonbootcamp/day_4
/content/pythonbootcamp/day_4
/content/pythonbootcamp/day_4


If you open the `Files` menu (left panel, folder icon), you'll see that the `merged.csv` file is now available.
___
**CAUTION**: Files that you save while using Colab are not retained after you close the notebook, as they only exist in Colab's temporary **session storage**. If you generate files and wish to keep them, make sure to download your files (with the same three dots menu) before you exit Colab.
___

That's it for our introduction to `pandas`! We hope you're starting to get a feel for how immensely powerful `pandas` can be. (Perhaps too powerful for a single bootcamp lecture...)

* To learn more about DataFrame methods, click here for the DataFrame documentation.
* To learn more about Series methods, click here for the DataFrame documentation.
* To learn about other common `pandas` routines, refer to the official `pandas` cookbook [here](https://pandas.pydata.org/docs/user_guide/cookbook.html). (No pandas were harmed in the making of this cookbook.)


# << `Exercises` >>

⏸ **Exercise 3a**: Just like with arrays, we can use the `.isna()` method to detect missing values in a DataFrame. Use this to identify all missing values in `clinvar`.

In [132]:
### write your code below ###
clinvar.isna()
clinvar.isna().any()
clinvar.isna().all()
clinvar.isna()
clinvar.head()
clinvar.isna()

,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
0,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
747773,False,False,False,False,False,False,False,False,False,False
747774,False,False,False,False,False,False,False,False,False,False
747775,False,False,False,False,False,False,False,False,False,False
747776,False,False,False,False,False,False,False,False,False,False


⏸ **Exercise 3b** Let's chain together some `pandas` methods! Which columns of `clinvar` are missing values?

*Hint*: The `.any()` method returns a `True` if any of the values in the Series or array are `True`.

In [189]:
### write your code below ###
clinvar.isna().any()
clinvar.isna().any(axis = 0)

,0
chromosome,False
position,False
reference,False
alternative,True
clinical_significance,True
ID,True
gene,True
molecular_consequence,True
disease,True
pathogenic_indicator,False


⏸ **Exercise 3c**: Find the total number of missing values in each column of `clinvar`.

*Hint*: Recall that `False` Booleans are considered equivalent to `0`, and `True` values are considered equivalent to `1`.

In [191]:
### write your code below ###
#clinvar.isna().any() == True
print(clinvar.isna().sum(axis=0))
#print(clinvar.isna().any(axis=0).sum())
#print(clinvar.isna().any().sum())


chromosome                    0
position                      0
reference                     0
alternative                 599
clinical_significance       571
ID                       147108
gene                        214
molecular_consequence     14637
disease                     571
pathogenic_indicator          0
dtype: int64
6


⏸ **Exercise 3d**: Which rows in `clinvar` are incomplete? Filter `clinvar` to show these rows.

*Hint*: You can change the axis along which `.any()` operates.

In [194]:
### write your code below ###
clinvar.isna().any(axis=1)
filter_mask_with_na = clinvar.isna().any(axis=1)
filter_mask_with_na = (clinvar.isna().any(axis=1) == True)
clinvar[filter_mask_with_na]

,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
0,1,930188,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
1,1,930203,C,T,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
3,1,930275,T,G,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
4,1,930336,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
5,1,931039,A,G,Uncertain_significance,NaN,SAMD11,synonymous_variant,not_provided,0
...,...,...,...,...,...,...,...,...,...,...
747758,22,50730998,C,A,Pathogenic,NaN,SHANK3,missense_variant,22q13.3_deletion_syndrome,1
747762,22,50731088,C,T,Likely_pathogenic,NaN,SHANK3,nonsense,not_provided,0
747763,22,50731118,T,C,Likely_pathogenic,NaN,SHANK3,missense_variant,not_provided,0
747767,22,50731197,TAGAA,T,Uncertain_significance,NaN,SHANK3,frameshift_variant,22q13.3_deletion_syndrome,0


⏸ **Exercise 4a** We're now going to move to data cleaning.

First, select rows in ```clinvar``` that have complete data (no missing values): save this filtered DataFrame to a variable called `clinvar_complete`. What are the dimensions of `clinvar_complete`?

In [235]:
### write your code below ###
complete = ~ clinvar.isna().any(axis=1)
complete =  (clinvar.isna().any(axis=1) == False)
clinvar_complete = clinvar[complete]
clinvar.isna().any(axis=1)

,0
0,True
1,True
2,False
3,True
4,True
...,...
747773,False
747774,False
747775,False
747776,False


In [152]:
# use this cell to check your work
clinvar_complete.isna().any()

,0
chromosome,False
position,False
reference,False
alternative,False
clinical_significance,False
ID,False
gene,False
molecular_consequence,False
disease,False
pathogenic_indicator,False


⏸ **Exercise 4b**: It seems like a significant number of `clinvar` rows contain missing values. Calculate the mean, median, and maximum number of missing values for `clinvar` rows.

*Hint*: This exercise uses the same principle as Exercise 3c.

In [154]:
clinvar.head()

,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
0,1,930188,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
1,1,930203,C,T,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
2,1,930248,G,A,Likely_benign,rs41285790,SAMD11,missense_variant,not_provided,0
3,1,930275,T,G,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
4,1,930336,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0


In [239]:
### write your code below ###
print(clinvar.isna().sum(axis=1))
print(clinvar.isna().sum(axis=1).mean())
print(clinvar.isna().sum(axis=1).median())


0         1
1         1
2         0
3         1
4         1
         ..
747773    0
747774    0
747775    0
747776    0
747777    0
Length: 747778, dtype: int64
0.21891523954970593
0.0


⏸ **Exercise 4c**: The `.dropna()` method accomplishes what we did with Boolean filtering, but it allows for additional flexibility when filtering rows. Look up the documentation for `.dropna()`, then use it to select rows in `clinvar` that have no more than 2 NA values. Assign this DataFrame to `clinvar_partial`.

In [248]:
### write your code below ###thresh: require that many non-NA value, 10 columns totally
print(clinvar.dropna(thresh=8))

clinvar.dropna(thresh=2)
clinvar_partial = clinvar.dropna(thresh=8)
print(clinvar_partial.isna().sum(axis=1).max())

        chromosome  position reference alternative   clinical_significance  \
0                1    930188         G           A  Uncertain_significance   
1                1    930203         C           T  Uncertain_significance   
2                1    930248         G           A           Likely_benign   
3                1    930275         T           G  Uncertain_significance   
4                1    930336         G           A  Uncertain_significance   
...            ...       ...       ...         ...                     ...   
747773          22  50738306         C           T                  Benign   
747774          22  50744057         A           G                  Benign   
747775          22  50744827         A           G                  Benign   
747776          22  50769034         G           T                  Benign   
747777          22  50769106         G           A           Likely_benign   

                 ID    gene molecular_consequence              

⏸ **Exercise 4d**: How many rows remain compared to `clinvar_complete` and the original `clinvar` DataFrame?

In [242]:
### write your code below ###
print(clinvar_partial.shape)
print(clinvar_complete.shape)
print(clinvar.shape)

(747607, 10)
(585977, 10)
(747778, 10)


⏸ **Challenge**: Create a new DataFrame called `pathogenic` that contains only pathogenic variants.

*Hint*: The `clinical_significance` column contains several different strings with the substring `'pathogenic'`. How can you take advantage of this to filter `clinvar`? If you're not sure, check the morning notebook.)

In [251]:
### write your code below ###
print(clinvar['clinical_significance'].str.contains('pathogenic'))
#m = clinvar['clinical_significance'].str.contains('pathogenic', na=False)
#m = clinvar['clinical_significance'].str.lower().str.contains('pathogenic', na=False)

m = clinvar_complete['clinical_significance'].str.lower().str.contains('pathogenic')


clinvar_complete[m]
#pathogenic = clinvar[clinvar['clinical_significance'].str.contains('pathogenic', na=False)]
pathogenic = clinvar_complete[m]
pathogenic.head()

0         False
1         False
2         False
3         False
4         False
          ...  
747773    False
747774    False
747775    False
747776    False
747777    False
Name: clinical_significance, Length: 747778, dtype: object


,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
125,1,1014143,C,T,Pathogenic,rs786201005,ISG15,nonsense,Immunodeficiency_38_with_basal_ganglia_calcifi...,1
138,1,1014316,C,CG,Pathogenic,rs672601345,ISG15,frameshift_variant,Immunodeficiency_38_with_basal_ganglia_calcifi...,1
141,1,1014359,G,T,Pathogenic,rs672601312,ISG15,nonsense,Immunodeficiency_38_with_basal_ganglia_calcifi...,1
159,1,1020239,G,C,Conflicting_interpretations_of_pathogenicity,rs201073369,AGRN,missense_variant,"Myasthenic_syndrome,_congenital,_8|not_specifi...",0
172,1,1022225,G,A,Pathogenic,rs756623659,AGRN,missense_variant,Congenital_myasthenic_syndrome,1


Find the number of pathogenic variants for each of three genes: `BRCA1`, `PAH`, and `CFTR`.

In [257]:
### write your code below ###
#print(pathogenic['gene'] == 'BRCA1')
#BRCA1 = pathogenic[pathogenic['gene'] == 'BRCA1']
#print(BRCA1)
#BRCA1.describe()

pathogenic[pathogenic['gene'].isin(['BRCA1','PAH', 'CFTR'])]

#value_counts = pathogenic[pathogenic['gene'].isin(['BRCA1','PAH', 'CFTR'])]['gene'].value_counts()

value_counts = pathogenic[pathogenic['gene'].isin(['BRCA1','PAH', 'CFTR'])]['gene'].value_counts()

value_counts_2 = pathogenic[pathogenic['gene'].isin(['BRCA1','PAH', 'CFTR'])].value_counts('gene')

print(value_counts)
print(value_counts_2)
pathogenic[pathogenic['gene'].isin(['BRCA1','PAH', 'CFTR'])]

gene
BRCA1    3201
CFTR      814
PAH       433
Name: count, dtype: int64
gene
BRCA1    3201
CFTR      814
PAH       433
Name: count, dtype: int64


,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
302049,7,117480081,GCGCCCGAGAGACCATGCAGAGGT,G,Pathogenic,rs397508136,CFTR,frameshift_variant,Cystic_fibrosis,1
302053,7,117480095,A,G,Pathogenic,rs397508328,CFTR,missense_variant,Cystic_fibrosis|not_provided,1
302054,7,117480096,T,A,Pathogenic,rs397508476,CFTR,missense_variant,Cystic_fibrosis,1
302055,7,117480096,T,C,Pathogenic/Likely_pathogenic,rs397508476,CFTR,missense_variant,Cystic_fibrosis,0
302056,7,117480096,T,G,Pathogenic,rs397508476,CFTR,missense_variant,Cystic_fibrosis,1
...,...,...,...,...,...,...,...,...,...,...
638399,17,43124106,T,G,Conflicting_interpretations_of_pathogenicity,rs748057929,BRCA1,5_prime_UTR_variant,"Breast-ovarian_cancer,_familial_1|Hereditary_c...",0
638415,17,43124112,T,C,Conflicting_interpretations_of_pathogenicity,rs777262055,BRCA1,5_prime_UTR_variant,Hereditary_breast_and_ovarian_cancer_syndrome|...,0
638433,17,43124117,T,C,Pathogenic,rs886040902,BRCA1,splice_acceptor_variant,"Breast-ovarian_cancer,_familial_1",1
638438,17,43124120,A,G,Conflicting_interpretations_of_pathogenicity,rs273898671,BRCA1,intron_variant,"Breast-ovarian_cancer,_familial_1|Hereditary_c...",0


# [Optional] More about `pandas`

Are you excited about `pandas`? Like, really excited? Here's some extra info on techniques that may be useful to you in the future. (We're not including this in the main lecture, as there's already a ton to go through!)

We'll focus on the `merged` dataset, which we derived by merging together `alleles` and `clinvar`. All of the rows in `merged` represent alleles found in both datasets.

First, let's quickly inspect a couple of different things, just to get our bearings.

In [233]:
# try it out:
# print the # of rows and cols in merged
merged.shape
# just to review how many alleles in alleles are also found in clinvar
merged.head()
alleles.head()
clinvar.head()


,chromosome,position,reference,alternative,clinical_significance,ID,gene,molecular_consequence,disease,pathogenic_indicator
0,1,930188,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
1,1,930203,C,T,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
2,1,930248,G,A,Likely_benign,rs41285790,SAMD11,missense_variant,not_provided,0
3,1,930275,T,G,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0
4,1,930336,G,A,Uncertain_significance,NaN,SAMD11,missense_variant,not_provided,0


In [ ]:
# try it out:
# view the first 10 rows of merged


Let's quickly look at the column labels, just so we can have a list of the columns we might want to examine.

In [ ]:
# try it out:
# examine the column labels: how many are there, and what are they?


Great! We can see that we have a good number of columns that derive from `clinvar`. There are also some duplicate columns, which are marked with the `_x` suffix to indicate that they derive from `alleles`, or the `_y` suffix to indicate that they derive from `clinvar`. You can adjust these suffixes using parameters in `pd.merge()`.

Let's examine some of the columns that may hold interesting categorical values:

In [ ]:
# try it out:
# find the unique values in the clinical_significance column


In [ ]:
# try it out:
# find the unique values in the molecular_consequence column


Given that not everyone will be versed in disease genetics or molecular biology, here's a quick crash course on what these mean.

*clinical_significance* refers to the observed effect of the mutation on human health. This data is curated by clinicians and uploaded to the ClinVar database.

*molecular_consequence* refers to the consequence of a specific mutation on the gene product (protein) of the mutated gene. Not all mutations yield the same molecular_consequence, and some consequences are more dire than others: for example, *nonsense mutations* can result in a gene product that may not perform its biological function.

## Fun tricks: rolling window operations

This is a fun feature of `pandas` that may be interesting to people who work with longitudinal-type data, such as time series data or genomic coordinate-related data. It is *not* a must-know feature of `pandas`.

The `.rolling()` method for DataFrames takes a `window` parameter, which specifies the number of rows to use to create a *rolling (sliding) window* across the DataFrame. This can be used to methodically calculate summaries such as sliding window averages using a `for` loop. (Yes, this is a very specific trick!)

In [ ]:
# calculating a rolling window average of allele frequencies on the 21st chromosome
# using the alleles dataset, sorted by genomic position on the 21st chromosome

rolling_mean = []
for window in alleles[alleles['chromosome'] == 21].sort_values('position').rolling(window = 500):
  rolling_mean.append(window['AF'].mean()) # get the average allele freq in the rolling wondow

In [ ]:
# plotting the rolling mean of allele frequencies across chromosome 21
# plt.plot takes two arrays/lists: the first corresponds to x values, the second to y values
# below, we created "dummy" x values that match the length of rolling_mean
# (there are more robust ways to plot this rolling mean :) )

plt.plot(np.arange(0, len(rolling_mean)), rolling_mean)

Want to learn more fun `pandas` tricks? It's time for you to move on to the full documentation [here](https://pandas.pydata.org/docs/user_guide/index.html#user-guide). Go wild!